In [1]:
%pylab inline
from papers import *

Populating the interactive namespace from numpy and matplotlib


In [2]:
pl, ts = arxiv_reading()
with open("./list_pkls/"+ts.split(",")[-1].strip().replace(" ", "_")+"_paper_list.pkl", "wb") as f:
    pck.dump(pl, f)
    
lp_c = ListPapers(pl)

New submissions for Fri,  6 Nov 20
Fetching...
Done...


In [3]:
st = lp_c.summary()

Number of papers: 51
Subjects: 
['Astrophysics of Galaxies (astro-ph.GA)', 'High Energy Astrophysical Phenomena (astro-ph.HE)', 'High Energy Physics - Phenomenology (hep-ph)', 'Solar and Stellar Astrophysics (astro-ph.SR)', 'Cosmology and Nongalactic Astrophysics (astro-ph.CO)', 'Instrumentation and Methods for Astrophysics (astro-ph.IM)', 'Plasma Physics (physics.plasm-ph)', 'General Relativity and Quantum Cosmology (gr-qc)', 'Earth and Planetary Astrophysics (astro-ph.EP)', 'High Energy Physics - Theory (hep-th)', 'Space Physics (physics.space-ph)']
Key words: 

Exclude key words: 

Total scores:



In [4]:
lp_c.add_key_words(["Chemistry", "Galactic center", "SNR", "Supernova Remnant", "MHD", "turbulence", \
                   "central parsec", "Sgr A", "astrochemistry", "CGM", "OVI", "OVII", "OVIII"])
lp_c.add_key_words("Cosmic Ray", boost=0.5)
lp_c.summary()


Number of papers: 51
Subjects: 
['Astrophysics of Galaxies (astro-ph.GA)', 'High Energy Astrophysical Phenomena (astro-ph.HE)', 'High Energy Physics - Phenomenology (hep-ph)', 'Solar and Stellar Astrophysics (astro-ph.SR)', 'Cosmology and Nongalactic Astrophysics (astro-ph.CO)', 'Instrumentation and Methods for Astrophysics (astro-ph.IM)', 'Plasma Physics (physics.plasm-ph)', 'General Relativity and Quantum Cosmology (gr-qc)', 'Earth and Planetary Astrophysics (astro-ph.EP)', 'High Energy Physics - Theory (hep-th)', 'Space Physics (physics.space-ph)']
Key words: 
['Chemistry', 'Galactic center', 'SNR', 'Supernova Remnant', 'MHD', 'turbulence', 'central parsec', 'Sgr A', 'astrochemistry', 'CGM', 'OVI', 'OVII', 'OVIII', 'Cosmic Ray']
Exclude key words: 

Total scores:
[1.4020765917518649, 0.0, 0.0, 0.0, 0.0, 0.0, 3.3574393492090961, 0.0, 0.0, 1.2186745567946178, 1.1819550379674872, 0.0, 1.6147029482237598, 0.0, 0.0, 0.0, 0.0, 0.0, 2.9762451140528454, 0.0, 0.0, 2.2072215147411272, 7.53083

In [5]:
lp_c.head()

ID: arXiv:2011.02589
Title: Evidence for galaxy quenching in the green valley caused by a lack of a  circumgalactic medium
Authors: Glenn G. Kacprzak; Nikole M. Nielsen; Hasti Nateghi; Christopher W. Churchill; Stephanie K. Pointon; Themiya Nanayakkara; Sowgat Muzahid; Jane C. Charlton
Date: New submissions for Fri,  6 Nov 20
Abstract: The relationship between a galaxy's properties and its circumgalactic medium
(CGM) provides a unique view of how galaxies evolve. We present an interesting
edge-on (i=86 degrees) disk galaxy (G1547) where the CGM is probed by a
background quasar at a distance of 84 kpc and within 10 degrees of the galaxy
major axis. G1547 does not have any detectable CGM absorption down to stringent
limits, covering HI (EW$_r$<0.02A, log(N(HI)/cm$^{-2}$)<12.6) and a range of
low and high ionisation absorption lines (OI, CII, NII, SiII, CIII, NIII,
SiIII, CIV, SiIV, NV and OVI). This system is rare, given the covering fraction
of $1.00_{-0.04}^{+0.00}$ for sub-L* galaxies

In [6]:
lp_d = lp_c.filter_subjects('Earth and Planetary Astrophysics (astro-ph.EP)', exclude=True)
lp_d.summary()

Number of papers: 48
Subjects: 
['Astrophysics of Galaxies (astro-ph.GA)', 'High Energy Astrophysical Phenomena (astro-ph.HE)', 'High Energy Physics - Phenomenology (hep-ph)', 'Solar and Stellar Astrophysics (astro-ph.SR)', 'Cosmology and Nongalactic Astrophysics (astro-ph.CO)', 'Instrumentation and Methods for Astrophysics (astro-ph.IM)', 'Plasma Physics (physics.plasm-ph)', 'General Relativity and Quantum Cosmology (gr-qc)', 'High Energy Physics - Theory (hep-th)', 'Space Physics (physics.space-ph)']
Key words: 
['Chemistry', 'Galactic center', 'SNR', 'Supernova Remnant', 'MHD', 'turbulence', 'central parsec', 'Sgr A', 'astrochemistry', 'CGM', 'OVI', 'OVII', 'OVIII', 'Cosmic Ray']
Exclude key words: 

Total scores:
[1.3426832228442203, 0.0, 0.0, 0.0, 0.0, 0.0, 3.3027309184174323, 0.0, 0.0, 1.1674838443768234, 1.1323909646270003, 0.0, 1.5456370547578517, 0.0, 0.0, 0.0, 2.9236424873317044, 0.0, 0.0, 2.1117306239734006, 7.3401038668642489, 1.1543046909949533, 0.0, 0.0, 0.0, 0.0, 1.11475

In [7]:
sigp_n = np.array(lp_d.tot_scores)[np.array(lp_d.tot_scores)>0.].size

In [8]:
lp_d.head(sync=True, n=sigp_n)

ID: arXiv:2011.02589
Title: Evidence for galaxy quenching in the green valley caused by a lack of a circumgalactic medium
Authors: Kacprzak, Glenn G.; Nielsen, Nikole M.; Nateghi, Hasti; Churchill, Christopher W.; Pointon, Stephanie K.; Nanayakkara, Themiya; Muzahid, Sowgat; Charlton, Jane C.
Date: 2020/11/05
Abstract: The relationship between a galaxy's properties and its circumgalactic medium
(CGM) provides a unique view of how galaxies evolve. We present an interesting
edge-on (i=86 degrees) disk galaxy (G1547) where the CGM is probed by a
background quasar at a distance of 84 kpc and within 10 degrees of the galaxy
major axis. G1547 does not have any detectable CGM absorption down to stringent
limits, covering HI (EW$_r$<0.02A, log(N(HI)/cm$^{-2}$)<12.6) and a range of
low and high ionisation absorption lines (OI, CII, NII, SiII, CIII, NIII,
SiIII, CIV, SiIV, NV and OVI). This system is rare, given the covering fraction
of $1.00_{-0.04}^{+0.00}$ for sub-L* galaxies within 50-100 kp

ID: arXiv:2011.02783
Title: The GALAH survey: Milky Way disc metallicity and alpha-abundance trends in combined APOGEE-GALAH catalogues
Authors: Nandakumar, Govind; Hayden, Michael R.; Sharma, Sanjib; Buder, Sven; Asplund, Martin; Bland-Hawthorn, Joss; De Silva, Gayandhi M.; D'Orazi, Valentina; Freeman, Ken C.; Kos, Janez; Lewis, Geraint F.; Martell, Sarah L.; Schlesinger, Katharine J.; Lin, Jane; Simpson, Jeffrey D.; Zucker, Daniel B.; Zwitter, Tomaz; Nordlander, Thomas; Casagrande, Luca; Lind, Karin; Cotar, Klemen; Stello, Dennis; Wittenmyer, Robert A.; Tepper-Garcia, Thor
Date: 2020/11/05
Abstract: GALAH and APOGEE are two high resolution multi object spectroscopic surveys
that provide fundamental stellar parameters and multiple elemental abundance
estimates for $>$ 400,000 stars in the Milky Way. They are complimentary in
both sky coverage and wavelength regime. Thus combining the two surveys will
provide us a large sample to investigate the disc metallicity and alpha
abundance tre

ID: arXiv:2011.02825
Title: Constraining the geometry of millisecond pulsars through simultaneous radio and gamma-ray light-curve fitting
Authors: Benli, O.; Petri, J.; Mitra, Dipanjan
Date: 2020/11/05
Abstract: Since the launch of the Fermi Gamma-Ray Space Telescope, several hundred of
gamma-ray pulsars have been discovered, some being radio-loud and some
radio-quiet with time-aligned radio and gamma-ray light-curves. In the
$2^\mathrm{nd}$ Fermi Pulsar Catalogue (2PC), 117 new gamma-ray pulsars have
been reported by using three years of data collected by the Large Area
Telescope (LAT) on the Fermi satellite, providing a wealth of information such
as the peak separation~$Δ$ of the gamma-ray pulsations and the radio
lag~$δ$ between the gamma-ray and radio pulses. We selected several
radio-loud millisecond gamma-ray pulsars with period~$P$ in the range 2-6~ms
and showing double peak in their gamma-ray profiles. We attempted to constrain
the geometry of their magnetosphere, namely the ma

ID: arXiv:2011.02717
Title: Kilonova Emission From Black Hole-Neutron Star Mergers. II. Luminosity Function and Implications for Target-of-opportunity Observations of Gravitational-wave Triggers and Blind Searches
Authors: Zhu, Jin-Ping; Wu, Shichao; Yang, Yuan-Pei; Zhang, Bing; Gao, He; Yu, Yun-Wei; Li, Zhuo; Cao, Zhoujian; Liu, Liang-Duan; Huang, Yan; Zhang, Xing-Han
Date: 2020/11/05
Abstract: We present detailed simulations of kilonova (KN) and GRB afterglow emission
originating from BH-NS mergers. We present KN luminosity function and discuss
the detectability of KN and GRB afterglow in connection with GW detections,
GW-triggered ToO observations, and blind searches in time-domain survey
observations. The predicted absolute magnitude of the BH-NS merger KNe at
$0.5\,{\rm day}$ after the merger falls in the range of $[-10,-15.5]$. The
simulated luminosity function contains the viewing angle distribution
information of the anisotropic KN emission. We simulate the GW detection rates,


In [9]:
print(lp_c._boost)

{'chemistry': 1.0, 'galactic center': 1.0, 'snr': 1.0, 'supernova remnant': 1.0, 'mhd': 1.0, 'turbulence': 1.0, 'central parsec': 1.0, 'sgr a': 1.0, 'astrochemistry': 1.0, 'cgm': 1.0, 'ovi': 1.0, 'ovii': 1.0, 'oviii': 1.0, 'cosmic ray': 0.5}


In [10]:
dir(lp_d[0])

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slotnames__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_authors_in_line',
 '_find_paper',
 '_read_page',
 '_url_prefix',
 'abstract',
 'arxiv_id',
 'authors',
 'comments',
 'customized_fields',
 'date',
 'fetch_pdf',
 'link',
 'link_ads',
 'nap_interval',
 'search_online',
 'subjects',
 'title']

In [11]:
lp_d.tot_score

AttributeError: 'ListPapers' object has no attribute 'tot_score'